## Prostate cancer study

Starting with the gene names corresponding to a protein list obtained by comparing 
patients whose disease progressed vs stable.

We first obtain a clean/valid human gene list we can use downstream.

In [3]:
genes = """...""" # Paste comma separated gene list here, the original list is not made public

In [2]:
gene_names = [gene_name.strip() for gene_name in genes.split(',')]

In [ ]:
from indra.databases import hgnc_client

clean_gene_names = []
hgnc_ids = []
for gene_name in gene_names:
    hgnc_id = hgnc_client.get_hgnc_id(gene_name)
    if not hgnc_id:
        hgnc_id = hgnc_client.get_current_hgnc_id(gene_name)
        if hgnc_id and hgnc_id != '5728':
            clean_gene_names.append(hgnc_client.get_hgnc_name(hgnc_id))
            hgnc_ids.append(hgnc_id)
    else:
        clean_gene_names.append(gene_name)
        hgnc_ids.append(hgnc_id)
clean_gene_names

In [ ]:
' '.join(clean_gene_names)

In [ ]:
hgnc_ids

We now obtain an INDRA subnetwork for a list of genes

In [ ]:
import requests

def query_indra_subnetwork(groundings):
    """Return a list INDRA subnetwork relations based on a list of groundings."""
    res = requests.post(
        'https://discovery.indra.bio/api/indra_subnetwork_relations',
        json={'nodes': groundings}
    )
    return res.json()

groundings = []
for hgnc_id in hgnc_ids:
    groundings.append(('HGNC', hgnc_id))
subnetwork_relations = query_indra_subnetwork(groundings)

In [ ]:
len(subnetwork_relations)

We first create an HTML Statement dump to be able to browse and curate statements

In [ ]:
import json
from indra.statements import stmts_from_json

# Gather statistics for HTML presentation
unique_stmts = {entry['data']['stmt_hash']: json.loads(entry['data']['stmt_json'])
                for entry in subnetwork_relations}
ev_counts_by_hash = {entry['data']['stmt_hash']: entry['data']['evidence_count']
                     for entry in subnetwork_relations}
source_counts_by_hash = {entry['data']['stmt_hash']: json.loads(entry['data']['source_counts'])
                         for entry in subnetwork_relations}
stmts = stmts_from_json(list(unique_stmts.values()))

from indra.assemblers.html import HtmlAssembler
ha = HtmlAssembler(stmts,
                   title='INDRA subnetwork statements',
                   db_rest_url='https://db.indra.bio',
                   ev_counts=ev_counts_by_hash,
                   source_counts=source_counts_by_hash)
html_str = ha.make_model()
ha.save_model('statements2.html')

We next upload the obtained network to NDEx

In [ ]:
import ndex2.client
from ndex2 import create_nice_cx_from_server
from indra.assemblers.cx import NiceCxAssembler
from indra.databases import ndex_client

ca = NiceCxAssembler(stmts, 'INDRA May Institute Network')
ncx = ca.make_model(self_loops=False)

style_network_id = '058c452f-b0d6-11ea-a4d3-0660b7976219'
style_ncx = create_nice_cx_from_server(
    server='http://test.ndexbio.org',
    uuid=style_network_id)
ncx.apply_style_from_network(style_ncx)

username, password = ndex_client.get_default_ndex_cred(ndex_cred=None)
ndex_args = {'server': 'http://public.ndexbio.org',
             'username': username,
             'password': password}
network_url = ncx.upload_to(**ndex_args)
network_url